In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import re

#Function to split text into sentences by fullstop(.)
'''def read_article(text):
    
    article = text.split(". ")
    sentences =[]
    
    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]"," ").split(" "))
    
    return sentences'''

# Read the text and tokenize into sentences
def read_article(text):
    
    sentences =[]
    
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]"," ")

    return sentences
    

# Create vectors and calculate cosine similarity b/w two sentences
def sentence_similarity(sent1,sent2,stopwords=None):
    if stopwords is None:
        stopwords = []
    
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    #build the vector for the first sentence
    for w in sent1:
        if not w in stopwords:
            vector1[all_words.index(w)]+=1
    
    #build the vector for the second sentence
    for w in sent2:
        if not w in stopwords:
            vector2[all_words.index(w)]+=1
            
    return 1-cosine_distance(vector1,vector2)

# Create similarity matrix among all sentences
def build_similarity_matrix(sentences,stop_words):
    #create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1!=idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
                
    return similarity_matrix


# Generate and return text summary
def generate_summary(text,top_n):
    
    nltk.download('stopwords')
    nltk.download('punkt')
    
    stop_words = stopwords.words('english')
    summarize_text = []
    
    # Step1: read text and tokenize
    sentences = read_article(text)
    
    # Steo2: generate similarity matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences,stop_words)
    
    # Step3: Rank sentences in similarirty matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    
    #Step4: sort the rank and place top sentences
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    
    # Step 5: get the top n number of sentences based on rank    
    for i in range(top_n):
        summarize_text.append(ranked_sentences[i][1])
    
    # Step 6 : outpur the summarized version
    return " ".join(summarize_text),len(sentences)

In [2]:
doc = """It is, perhaps, dreadfully apt that an invasion which began 20 years ago as a counter-terrorism operation has ended in the horror of a mass casualty terrorist attack. The US-led attempt to destroy al-Qaida and rescue Afghanistan from the Taliban was undercut by the Iraq war, which spawned Islamic State. Now the circle is complete as an Afghan IS offshoot emerges as America’s new nemesis.
The Kabul airport atrocity shows just how difficult it is to break the cycle of violence, vengeance and victimisation. Joe Biden’s swift vow to hunt down the perpetrators and “make them pay” presumably means US combat forces will again be in action in Afghanistan soon. If the past is any guide, mistakes will be made, civilians will die, local communities will be antagonised. Result: more terrorists.

It is an obvious irony that US military chiefs in Kabul are collaborating with the Taliban, their sworn enemy, against the common IS foe as the evacuation ends. This suggests negotiators, on both sides, could have tried harder to reach a workable peace deal. It may augur well for future cooperation, for example on humanitarian aid. But the Taliban has many faces – and many cannot be trusted.

Last week’s events have raised yet more questions about Biden’s judgment and competence. He will be blamed personally. His predicament recalls the downfall of another Democratic president, Jimmy Carter. After the disastrous failure of Operation Eagle Claw to rescue US hostages in Tehran in April 1980, Carter was voted out of office the following November.

Biden faces Republican calls to resign. His approval ratings have plunged. But he defiantly insists that quitting Afghanistan is the right thing to do. Polls suggest most Americans agree, though they are critical of how it has been managed. Unlike in Carter’s time, the next presidential election is three years away. By then the agony and humiliations of recent days may be a distant memory.

The Kabul debacle also casts doubt on Biden’s new counter-terrorism strategy, which reportedly downgrades the threat posed by Islamist terrorism to the US. His national security team wants to shift global priorities and resources to meet different, 21st-century challenges to American hegemony, such as China, cyberwarfare and the climate crisis.

Biden is said to want to use the 20th anniversary of the 11 September al-Qaida attacks on New York and Washington to declare America’s “forever wars” over – for which he will claim credit. Setting the Afghan shambles aside, he is expected to say the era of invasion, occupation, nation-building and the “global war on terror” is at an end. “The US approach should centre on gathering intelligence, training indigenous forces, and maintaining air power as well as special forces capabilities for the occasional strike when necessary,” foreign policy analysts Bruce Riedel and Michael O’Hanlon argued recently.

No one knows whether such a costly, hard to organise strategy will work in the long term. But the shift is already having tangible consequences. In Iraq, for example, US combat operations will cease in December. About 2,500 Americans will stay, to train and advise. In Syria, a small number of special forces will remain. Iraqis understandably worry about an IS comeback and an Afghan-style implosion. The same story of US disengagement and drawback is heard across the Middle East as the US “pivots” to Asia. Combat aircraft are being redeployed, carrier battlegroups may be reassigned to the Pacific theatre, and anti-missile batteries are being withdrawn from Iraq, Kuwait, Jordan and Saudi Arabia. Most of these assets were pointed at Iran, deemed a prime sponsor of terrorism.

In the Sahel, west Africa, the Democratic Republic of the Congo and Mozambique, the US barely registers in the fight against Boko Haram and assorted IS and al-Qaida affiliates. The impressively named US Africa Command is headquartered in Stuttgart. President Muhammadu Buhari warns that Nigeria could suffer a similar fate to Afghanistan without a “comprehensive partnership” with the US. “Some sense the west is losing its will for the fight,” he said. For US allies, all this points to a new era of enforced self-sufficiency and greater uncertainty. While Islamist-inspired attacks in the US have been rare since 9/11, in Europe many hundreds have died. Yet collective European counter-terrorism efforts often lack a military cutting edge. An exception was France’s ill-supported Operation Barkhane in Mali – until it was halted this year after suffering many casualties for little gain.

The chaos in Afghanistan has vividly dramatised the ongoing threat from international terrorism. With up to 10,000 foreign Islamist fighters in the country, according to the UN, fears grow it will again become a launchpad for global jihad. So the prospect of a less directly engaged, homeland-focused American counter-terrorism approach is alarming for partners dependent on US leadership and protection.

European Nato allies, sniping at Biden, are in denial. They don’t want to admit his Afghan withdrawal is just the start of something bigger. And as recent events painfully demonstrate, the UK is not remotely able to fend for itself."""

In [4]:
generate_summary(doc,top_n=5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('His national security team wants to shift global priorities and resources to meet different, 21st-century challenges to American hegemony, such as China, cyberwarfare and the climate crisis. In the Sahel, west Africa, the Democratic Republic of the Congo and Mozambique, the US barely registers in the fight against Boko Haram and assorted IS and al-Qaida affiliates. An exception was France’s ill-supported Operation Barkhane in Mali – until it was halted this year after suffering many casualties for little gain. The US-led attempt to destroy al-Qaida and rescue Afghanistan from the Taliban was undercut by the Iraq war, which spawned Islamic State. After the disastrous failure of Operation Eagle Claw to rescue US hostages in Tehran in April 1980, Carter was voted out of office the following November.',
 49)